# 라이브러리

In [143]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)


# 전처리

In [144]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [145]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [146]:
data$target<-(ifelse(data$취업직종대분류=="경영·행정·사무직","경영·행정·사무직",data$취업직종대분류))

data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직", "음식 서비스직", "미용·예식 서비스직", "영업·판매직", "스포츠·레크리에이션직","경호·경비직"), "서비스직", data$target)

data$target <- ifelse(grepl("^\\d+$", data$target), "기타", data$target)

In [147]:
data <- subset(data, data$target == "서비스직")
table(data$target)

In [149]:
data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직"), "청소 및 기타 개인서비스직", "서비스 및 판매 관련 직군")
table(data$target)

In [151]:
data <- subset(data, data$target == "서비스 및 판매 관련 직군")
table(data$target)


서비스 및 판매 관련 직군 
                    3259 

In [152]:
data$target<- ifelse(data$취업직종대분류=="음식 서비스직 ","음식 서비스직 ",data$취업직종대분류)
data$target <- ifelse(data$취업직종대분류 %in% c("미용·예식 서비스직", "영업·판매직"), "영업·미용·예식 서비스직", data$target)
data$target <- ifelse(data$취업직종대분류 %in% c("스포츠·레크리에이션직","경호·경비직"), "경호·경비·스포츠·레크리에이션직", data$target)
table(data$target)


경호·경비·스포츠·레크리에이션직         영업·미용·예식 서비스직 
                           1218                             926 
                  음식 서비스직 
                           1115 

In [153]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [154]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [155]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [156]:
#   근무지역 분류 바꾸기
data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

data_filtered$강원 <- data_filtered$근무지역분류 == "강원"

data_filtered$전남 <- data_filtered$근무지역분류 == "전남"

data_filtered$충북 <- data_filtered$근무지역분류 == "충북"

data_filtered$경기 <- data_filtered$근무지역분류 == "경기"

data_filtered$인천 <- data_filtered$근무지역분류 == "인천"

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"

data_filtered$울산 <- data_filtered$근무지역분류 == "울산"

data_filtered$대전 <- data_filtered$근무지역분류 == "대전"

data_filtered$부산 <- data_filtered$근무지역분류 == "부산"

data_filtered$전북 <- data_filtered$근무지역분류 == "전북"

data_filtered$광주 <- data_filtered$근무지역분류 == "광주"

data_filtered$충남 <- data_filtered$근무지역분류 == "충남"

data_filtered$대구 <- data_filtered$근무지역분류 == "대구"

data_filtered$경남 <- data_filtered$근무지역분류 == "경남"

data_filtered$제주 <- data_filtered$근무지역분류 == "제주"

data_filtered$세종 <- data_filtered$근무지역분류 == "세종"

data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

In [157]:
# 월
data_filtered$Jan <- data_filtered$월  == 01
data_filtered$Feb <- data_filtered$월  == 02
data_filtered$Mar <- data_filtered$월  == 03
data_filtered$Apr <- data_filtered$월  == 04
data_filtered$May <- data_filtered$월  == 05
data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09
data_filtered$Oct <- data_filtered$월  == 10
data_filtered$Nov <- data_filtered$월  == 11
data_filtered$Dec <- data_filtered$월  == 12

In [158]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	3259 obs. of  51 variables:
 $ 취업직종대분류   : chr  "음식 서비스직" "스포츠·레크리에이션직" "스포츠·레크리에이션직" "스포츠·레크리에이션직" ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 1 1 2 1 2 1 1 1 4 4 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 6 10 8 10 8 10 10 10 7 8 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 4 4 4 4 4 4 4 10 12 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 17 9 9 9 9 9 9 9 7 12 ...
 $ target           : chr  "음식 서비스직" "경호·경비·스포츠·레크리에이션직" "경호·경비·스포츠·레크리에이션직" "경호·경비·스포츠·레크리에이션직" ...
 $ age_20           : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30           : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE .

[1] 0

In [159]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	3259 obs. of  51 variables:
 $ 취업직종대분류   : chr  "음식 서비스직" "스포츠·레크리에이션직" "스포츠·레크리에이션직" "스포츠·레크리에이션직" ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 1 1 2 1 2 1 1 1 4 4 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 6 10 8 10 8 10 10 10 7 8 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 4 4 4 4 4 4 4 10 12 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 17 9 9 9 9 9 9 9 7 12 ...
 $ target           : chr  "음식 서비스직" "경호·경비·스포츠·레크리에이션직" "경호·경비·스포츠·레크리에이션직" "경호·경비·스포츠·레크리에이션직" ...
 $ age_20           : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30           : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE .

[1] 0

In [160]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [161]:
str(data_filtered_subset)
sum(is.na(data_filtered_subset))

'data.frame':	3259 obs. of  47 variables:
 $ 취업직종대분류: chr  "음식 서비스직" "스포츠·레크리에이션직" "스포츠·레크리에이션직" "스포츠·레크리에이션직" ...
 $ target        : chr  "음식 서비스직" "경호·경비·스포츠·레크리에이션직" "경호·경비·스포츠·레크리에이션직" "경호·경비·스포츠·레크리에이션직" ...
 $ age_20        : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30        : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FAL

[1] 0

In [162]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [163]:
str(data_filtered_subset)

'data.frame':	3259 obs. of  47 variables:
 $ 취업직종대분류: chr  "음식 서비스직" "스포츠·레크리에이션직" "스포츠·레크리에이션직" "스포츠·레크리에이션직" ...
 $ target        : chr  "음식 서비스직" "경호·경비·스포츠·레크리에이션직" "경호·경비·스포츠·레크리에이션직" "경호·경비·스포츠·레크리에이션직" ...
 $ age_20        : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30        : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FAL

In [164]:
summary(data_filtered_subset)

 취업직종대분류        target            age_20          age_30       
 Length:3259        Length:3259        Mode :logical   Mode :logical  
 Class :character   Class :character   FALSE:2158      FALSE:2837     
 Mode  :character   Mode  :character   TRUE :1101      TRUE :422      
   age_40          age_50          age_60          age_70       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:2939      FALSE:2812      FALSE:2569      FALSE:2980     
 TRUE :320       TRUE :447       TRUE :690       TRUE :279      
 시각장애_중증   시각장애_경증   지체장애_중증   지체장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:3198      FALSE:3035      FALSE:2900      FALSE:2408     
 TRUE :61        TRUE :224       TRUE :359       TRUE :851      
 지적장애_경증   지적장애_중증   청각장애_경증   청각장애_중증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:3255      FALSE:1958      FALSE:3093      FALSE:3115     
 TRUE :4         TRUE :1301      TRUE :166       TRUE :1

# 변동사항

In [165]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [166]:
data_filtered_subset <- subset(data_filtered_subset, select = -c(취업직종대분류))

In [167]:
str(data_filtered_subset)

'data.frame':	3259 obs. of  46 variables:
 $ target       : Factor w/ 3 levels "경호·경비·스포츠·레크리에이션직",..: 3 1 1 1 1 1 1 1 1 1 ...
 $ age_20       : logi  TRUE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30       : logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증: logi  FALSE FALSE TRUE FALSE TRUE FALSE ...
 $ 지체장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ 정신장애_경증: logi  FALSE FALSE FALSE F

In [168]:
lm(target~.,data_filtered_subset)


Call:
lm(formula = target ~ ., data = data_filtered_subset)

Coefficients:
      (Intercept)         age_20TRUE         age_30TRUE         age_40TRUE  
          2.11316            0.63112            0.66969            0.52189  
       age_50TRUE         age_60TRUE         age_70TRUE  시각장애_중증TRUE  
          0.32653           -0.02237                 NA           -0.60014  
시각장애_경증TRUE  지체장애_중증TRUE  지체장애_경증TRUE  지적장애_경증TRUE  
         -0.45924           -0.91457           -0.47300            0.26197  
지적장애_중증TRUE  청각장애_경증TRUE  청각장애_중증TRUE  정신장애_경증TRUE  
          0.08292           -0.32814           -0.02924           -0.36943  
정신장애_중증TRUE           경북TRUE           강원TRUE           전남TRUE  
               NA           -0.25319           -0.25194           -0.27598  
         충북TRUE           경기TRUE           인천TRUE           서울TRUE  
         -0.21059           -0.24825           -0.33293           -0.34540  
         울산TRUE           대전TRUE           부산TRUE           전북TRUE  
     

In [169]:
summary(data_filtered_subset)

                             target       age_20          age_30       
 경호·경비·스포츠·레크리에이션직:1218   Mode :logical   Mode :logical  
 영업·미용·예식 서비스직        : 926   FALSE:2158      FALSE:2837     
 음식 서비스직                  :1115   TRUE :1101      TRUE :422      
   age_40          age_50          age_60          age_70       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:2939      FALSE:2812      FALSE:2569      FALSE:2980     
 TRUE :320       TRUE :447       TRUE :690       TRUE :279      
 시각장애_중증   시각장애_경증   지체장애_중증   지체장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:3198      FALSE:3035      FALSE:2900      FALSE:2408     
 TRUE :61        TRUE :224       TRUE :359       TRUE :851      
 지적장애_경증   지적장애_중증   청각장애_경증   청각장애_중증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:3255      FALSE:1958      FALSE:3093      FALSE:3115     
 TRUE :4         TRUE :1301      TRUE :166       TRUE :144      
 정신장애_경증  

# 시험준비

In [170]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [171]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [172]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [173]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=500)

In [174]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [175]:
# KNN 학습시키기 3,5,7,
# knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [176]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =20)

# weights:  983


initial  value 3181.012980 
iter  10 value 1685.019838
iter  20 value 1552.739634
iter  30 value 1408.462508
iter  40 value 1212.130168
iter  50 value 1039.650849
iter  60 value 910.788314
iter  70 value 830.588378
iter  80 value 768.209832
iter  90 value 737.288883
iter 100 value 720.664646
final  value 720.664646 
stopped after 100 iterations


# 검증하기

In [177]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [178]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [179]:
# KNN 검증하기
# mean(knnmodel==y) 

In [180]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [181]:
#  RF 예측력
mean(predrf == y)

[1] 0.7192623

In [182]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.6977459

In [183]:
#  KNN 예측력
# mean(knnmodel == y)

In [184]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.6793033

# 오분류표

In [185]:
#  Random Forest 오분류표
table(predrf,y)

                                 y
predrf                            경호·경비·스포츠·레크리에이션직
  경호·경비·스포츠·레크리에이션직                             297
  영업·미용·예식 서비스직                                      17
  음식 서비스직                                                51
                                 y
predrf                            영업·미용·예식 서비스직 음식 서비스직
  경호·경비·스포츠·레크리에이션직                      38            29
  영업·미용·예식 서비스직                             129            29
  음식 서비스직                                       110           276

In [186]:
#  SVM 오분류표
table(predmodelsvm,y)

                                 y
predmodelsvm                      경호·경비·스포츠·레크리에이션직
  경호·경비·스포츠·레크리에이션직                             291
  영업·미용·예식 서비스직                                      21
  음식 서비스직                                                53
                                 y
predmodelsvm                      영업·미용·예식 서비스직 음식 서비스직
  경호·경비·스포츠·레크리에이션직                      48            34
  영업·미용·예식 서비스직                             103            13
  음식 서비스직                                       126           287

In [187]:
#  KNN 오분류표
# table(y, knnmodel)

In [188]:
# nnet 오분류표
table(prednnet,y)

                                 y
prednnet                          경호·경비·스포츠·레크리에이션직
  경호·경비·스포츠·레크리에이션직                             281
  영업·미용·예식 서비스직                                      50
  음식 서비스직                                                34
                                 y
prednnet                          영업·미용·예식 서비스직 음식 서비스직
  경호·경비·스포츠·레크리에이션직                      35            42
  영업·미용·예식 서비스직                             151            61
  음식 서비스직                                        91           231

In [189]:
# 새로운 데이터에 대한 예측
predict(rf, (list(age_20=FALSE,age_30=FALSE,age_40=FALSE,age_50=FALSE,age_60=TRUE,age_70=FALSE,
            시각장애_중증=TRUE,시각장애_경증=FALSE,지체장애_경증=FALSE,지체장애_중증=FALSE,지적장애_경증=FALSE,지적장애_중증=FALSE,
            청각장애_경증=FALSE,청각장애_중증=FALSE,정신장애_경증=FALSE,정신장애_중증=FALSE,
            경북=TRUE,강원=FALSE,전남=FALSE,충북=FALSE,경기=FALSE,인천=FALSE,서울=FALSE,울산=FALSE,대전=FALSE,부산=FALSE,
            전북=FALSE,광주=FALSE,충남=FALSE,대구=FALSE,경남=FALSE,제주=FALSE,세종=FALSE,경북=FALSE,
            Jan=TRUE,Feb=FALSE,Mar=FALSE,Apr=FALSE,May=FALSE,Jun=FALSE,Jul=FALSE,Aug=FALSE,Sep=FALSE,
            Oct=FALSE,Nov=FALSE,Dec=FALSE)), type = "prob")

# pred_prob <- predict(rf, new_data, type = "prob")
# pred_prob <- predict(rf, new_data, type = "prob")

,경호·경비·스포츠·레크리에이션직,영업·미용·예식 서비스직,음식 서비스직
1,0.838,0.114,0.048
